## Data Collection


We will be using data from here: https://www.kaggle.com/ealaxi/paysim1

In [1]:
import pandas as pd

In [2]:
rawData = pd.DataFrame(pd.read_csv('Synthetic_Fraud_Data.csv'))
rawData.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


Dataset is synthetic, so there should be very little cleaning to do.

In [3]:
rawData.shape

(6362620, 11)

With over 6.3 million records, no further data needs to be added, which is good, because I couldn't find any more.

## Data Organization

Created a repository at https://github.com/NickLamm/SpringboardCapstone2

## Data Definition

In [4]:
print(rawData.columns)
rawData.describe()

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')


,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


###### Column definitions, from the kaggle page:

step - maps a unit of time in the real world. In this case 1 step is 1 hour of time. Total steps 744 (30 days simulation).

type - CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER.

amount - amount of the transaction in local currency.

nameOrig - customer who started the transaction

oldbalanceOrg - initial balance before the transaction

newbalanceOrig - new balance after the transaction

nameDest - customer who is the recipient of the transaction

oldbalanceDest - initial balance recipient before the transaction. Note that there is not information for customers that start with M (Merchants).

newbalanceDest - new balance recipient after the transaction. Note that there is not information for customers that start with M (Merchants).

isFraud - This is the transactions made by the fraudulent agents inside the simulation. In this specific dataset the fraudulent behavior of the agents aims to profit by taking control or customers accounts and try to empty the funds by transferring to another account and then cashing out of the system.

isFlaggedFraud - The business model aims to control massive transfers from one account to another and flags illegal attempts. An illegal attempt in this dataset is an attempt to transfer more than 200.000 in a single transaction.

##### Checking data 

Step should have 744 unique values. Type should have 5, and isFlaggedFraud/isFraud are binary so should be 0 or 1.

In [5]:
unq = rawData.apply(lambda col: len(col.unique()))
unq

step                  743
type                    5
amount            5316900
nameOrig          6353307
oldbalanceOrg     1845844
newbalanceOrig    2682586
nameDest          2722362
oldbalanceDest    3614697
newbalanceDest    3555499
isFraud                 2
isFlaggedFraud          2
dtype: int64

Good thing we checked, there are only 743 steps in this data.

Checking unique values for names:

In [6]:
print(f'Unique origin names: {unq["nameOrig"]}, {round(100*unq["nameOrig"]/len(rawData),2)}% of total records\n')
print(f'Unique destination names: {unq["nameDest"]}, {round(100*unq["nameDest"]/len(rawData),2)}% of total records')

Unique origin names: 6353307, 99.85% of total records

Unique destination names: 2722362, 42.79% of total records


While there are almost as many transactions as there are origin names, there are less than half as many destinations, most likely representing many individuals paying to larger financial entities.

In [7]:
rawData.dtypes

step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object

## Data Cleaning

Not much will have to be done, as I said before.

Type can be a category, fraud flags can be boolean.

In [8]:
rawData['type'] = rawData['type'].astype('category')
rawData['isFraud'] = rawData['isFraud'].astype('bool')
rawData['isFlaggedFraud'] = rawData['isFlaggedFraud'].astype('bool')
rawData.dtypes

step                 int64
type              category
amount             float64
nameOrig            object
oldbalanceOrg      float64
newbalanceOrig     float64
nameDest            object
oldbalanceDest     float64
newbalanceDest     float64
isFraud               bool
isFlaggedFraud        bool
dtype: object

In [9]:
rawData.isna().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

No missing values, due to coming from a simulation.

In [10]:
rawData.duplicated().sum()

0

No exact duplicates.

In [19]:
try:
    assert rawData[rawData['isFlaggedFraud']]['amount'].min() >= 200000
    assert rawData[rawData['amount']<200000]['isFlaggedFraud'].sum() == 0
except AssertionError: print('There are incorrect values in the \'isFlaggedFraud\' column.')
else: print('All transactions are correctly flagged based on amount.')

All transactions are correctly flagged based on amount.


In [12]:
flagCt = len(rawData[rawData['isFlaggedFraud']])
correctFlagsRatio = len(rawData[rawData['isFlaggedFraud'] & rawData['isFraud']])/(flagCt)
fraudCt = len(rawData[rawData['isFraud']])
print(f'Filter identified {flagCt} fraudulent transactions, {100*correctFlagsRatio}% of which were truly fraud.')
print(f'However, {fraudCt-(flagCt*correctFlagsRatio)} fraudulent transactions were not flagged')

Filter identified 16 fraudulent transactions, 100.0% of which were truly fraud.
However, 8197.0 fraudulent transactions were not flagged


'isFlaggedFraud' will likely be irrelavalent, especially since it's just a simple boolean condition. Depending on data exploration, it will probably be dropped.